In [ ]:
import os
import time
import gspread
from gspread.exceptions import APIError

def update_with_backoff(sheet_name, worksheet_name, start_row, start_col, num_updates, multiplier):
    """
    Updates cells in a Google Sheet with a backoff algorithm to handle rate limits.

    :param sheet_name: The name of the Google Sheet.
    :param worksheet_name: The name of the worksheet in the Google Sheet.
    :param start_row: The starting row for updates.
    :param start_col: The starting column for updates.
    :param num_updates: The number of cells to update.
    :param multiplier: The value to multiply for each update.
    """
    try:
        # Authenticate with Google Sheets API
        gc_filepath = os.path.expanduser('~google_secret.json')
        gc = gspread.service_account(filename=gc_filepath)
        sheet = gc.open(sheet_name)
        worksheet = sheet.worksheet(worksheet_name)

        for i in range(num_updates):
            row = start_row
            col = start_col + i
            value = multiplier * i

            retries = 100  # Maximum number of retries
            delay = 1    # Initial delay in seconds

            while retries > 0:
                try:
                    worksheet.update_cell(row, col, value)
                    print(f"Updated cell ({row}, {col}) with value {value}")
                    break  # Break if the update is successful
                except APIError as e:
                    error_message = str(e)
                    if "Quota exceeded" in error_message or "[429]" in error_message:
                        print(f"Quota exceeded. Retrying in {delay} seconds...")
                        time.sleep(delay)
                        delay *= 2  # Exponential backoff
                        retries -= 1
                    else:
                        raise  # Re-raise if it's a different error
            else:
                print(f"Failed to update cell ({row}, {col}) after multiple retries.")

        print("Cells updated successfully!")

    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function to update cells with backoff logic
update_with_backoff(
    sheet_name='',
    worksheet_name='',
    start_row=1,
    start_col=2,
    num_updates=300,
    multiplier=3.14
)


In [ ]:
#Test function for MP Updater
import time
from gspread.exceptions import APIError

def update_cell_with_backoff(update_function, retries=1000, initial_delay=1):
    """
    Updates a cell with exponential backoff for handling API rate limits.
    
    :param update_function: The update function to call (e.g., MP_WS.update_cell).
    :param retries: Number of retry attempts.
    :param initial_delay: Initial delay for backoff in seconds.
    """
    delay = initial_delay

    for attempt in range(retries):
        try:
            update_function()  # Call the update function passed as an argument
            print("Update successful!")
            return  # Exit if the update is successful
        except APIError as e:
            error_message = str(e)
            if "Quota exceeded" in error_message or "[429]" in error_message:
                print(f"Quota exceeded. Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2  # Exponential backoff
            else:
                raise  # Re-raise if it's a different error

    print(f"Failed to update cell after {retries} retries.")


In [ ]:
# Example of usage
update_cell_with_backoff(lambda: MP_WS.update_cell(row_value[7], column_value, swapped_tasks_3PL))
